## Load the IMDB dataset and create the vocabulary

In [1]:
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

EMBEDDING_DIM=50
VOCAB_SIZE=20000

# Get IMDB dataset
imdb = IMDB(split='train')

# Load English tokenizer, tagger, parser and NER
tokenizer = get_tokenizer('spacy', language='en')

# build the vocab
counter = Counter()
for i, (label, line) in enumerate(imdb):
    counter.update(tokenizer(line))

ordered_dict = OrderedDict(counter.most_common()[:VOCAB_SIZE])
vocab = vocab(ordered_dict)

# insert special tokens and set default index to 'unknown'
vocab.insert_token('<PAD>', 0)
vocab.insert_token('<UNK>', 1)
vocab.set_default_index(1)

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 25.8MB/s]


## Create embedding vectors from GloVe

In [2]:
import torchtext as text

# load glove embeddings
vec = text.vocab.GloVe(name='6B', dim=50)
# create the embedding matrix, a torch tensor in the shape (num_words+1, embedding_dim)
word_emb = vec.get_vecs_by_tokens(vocab.get_itos())

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399999/400000 [00:10<00:00, 39691.91it/s]


## Build up train/test dataset

In [7]:
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset
from sklearn.model_selection import train_test_split
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# transform input text and label to ids
def process_text(text):
    return vocab(tokenizer(text))

label_to_ids = {'pos':0, 'neg':1}

# preprocess a batch of raw data (string-like text/labels) into tensors
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_to_ids[_label])
        processed_text = torch.tensor(process_text(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    # label must be in the same size as target
    label_list = torch.tensor(label_list, dtype=torch.float)[:,None]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter, test_iter= IMDB()

# transform datasets iterator into map style so that they can be repeatedly loaded in a new epoch
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# further Splitting the val_dataset from the training dataset into 75% -> val_dataset ; 25% -> train dataset
val_dataset, test_dataset = train_test_split(test_dataset, test_size=0.25, random_state=99)

train_dataloader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=128,
                             shuffle=True, collate_fn=collate_batch)

# Question 1 # Write a validation dataloader by spliting the training data
val_dataloader =  DataLoader(val_dataset, batch_size=128,
                             shuffle=True, collate_fn=collate_batch) 

In [8]:
print(len(train_dataset))
print(len(test_dataset))
print(len(val_dataset))

25000
6250
18750


## Define the logistic regression model

In [18]:
# logistic model
import torch
import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module):
    def __init__(self, word_vec, embed_dim):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        
        # Question 2 : Replace the EmbeddingBag using PyTorch builtin functions that does the same job of computing sentence representation by taking average of the word embeddings.
        self.embedding = nn.EmbeddingBag(num_embeddings = len(vocab), 
                                         embedding_dim = embed_dim, 
                                         padding_idx = 0).from_pretrained(word_vec, freeze=False)
        #self.embedding = nn.EmbeddingBag(*(word_vec.size())).from_pretrained(word_vec, freeze=False)
                
        # Question 3 : Write a Fully Connected Layer (FC layer) with output size of 100 followed by a non-linear activation e.g., ReLU
        self.relu = torch.nn.ReLU()
        self.fc1 = nn.Linear(embed_dim, 100)

        self.fc2 = nn.Linear(100, 1)
        self._init_weights()

    def _init_weights(self):
        """Initialize network parameters 
        """
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        
        # Question 4: Use the new model you define in __init__()
        # embed = self.embedding(text)
        # fc = self.fc1(embed)
        # reLU = self.relu(fc)
        # x = self.fc2(reLU)

        # return torch.sigmoid(x)
        return torch.sigmoid(self.fc2(self.relu(self.fc1(embedded))))

## Define train and test function

In [19]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        # forward propagation
        predicted_label = model(text, offsets)
        # calculate loss and backpropagate to model paramters
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # update parameters by stepping the optimizer
        optimizer.step()
        total_acc += ((predicted_label > 0.5) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## Train and evaluate the model for several epochs

In [ ]:
# Hyperparameters
EPOCHS = 30 # epoch

model = LogisticRegression(word_vec=word_emb, embed_dim=EMBEDDING_DIM).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)
criterion = torch.nn.BCELoss()
total_accu = None

counter = 0
N = 10
ACC = -1

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    
    # Question 5: Use your validation set to early stop the model. Remember to early stop when the validation accuracy does not improve for continous N number of epochs where N is a hyperparameter. Set N = 10
    val_accuracy = evaluate(val_dataloader)
    print(f"Accuracy for validation is {val_accuracy}")
    if ACC >= val_accuracy:
      counter += 1
      print(counter)
      if counter == N:
        break;
    else:
      ACC = val_accuracy
      counter = 0

| epoch   1 |    50/  196 batches | accuracy    0.506
| epoch   1 |   100/  196 batches | accuracy    0.506
| epoch   1 |   150/  196 batches | accuracy    0.529
Accuracy for validation is 63.90474666666667
| epoch   2 |    50/  196 batches | accuracy    0.561
| epoch   2 |   100/  196 batches | accuracy    0.578
| epoch   2 |   150/  196 batches | accuracy    0.594
Accuracy for validation is 63.90474666666667
1
| epoch   3 |    50/  196 batches | accuracy    0.621
| epoch   3 |   100/  196 batches | accuracy    0.637
| epoch   3 |   150/  196 batches | accuracy    0.643
Accuracy for validation is 63.887146666666666
2
| epoch   4 |    50/  196 batches | accuracy    0.662
| epoch   4 |   100/  196 batches | accuracy    0.665
| epoch   4 |   150/  196 batches | accuracy    0.682
Accuracy for validation is 63.88010666666667
3
| epoch   5 |    50/  196 batches | accuracy    0.696
| epoch   5 |   100/  196 batches | accuracy    0.700
| epoch   5 |   150/  196 batches | accuracy    0.701
Acc

In [22]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.2f}%'.format(accu_test))

test accuracy    63.80%
